In [ ]:
import cv2
import os

In [ ]:
images = []
targets = []

In [ ]:
prefix_dir_without_mask = 'dataset/without_mask/'
prefix_dir_with_mask = 'dataset/with_mask/'

In [ ]:
content = os.listdir(prefix_dir_without_mask)
for image in content:
    try:
        image_path = prefix_dir_without_mask + image
        image = cv2.imread(image_path)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image_grey, (100, 100))
        images.append(resized_image)
        targets.append(1)
    except Exception as e:
        print("Exception occured as ",e)
    

In [ ]:
len(images)

1930

In [ ]:
content = os.listdir(prefix_dir_with_mask)
for image in content:
    try:
        image_path = prefix_dir_with_mask + image
        image = cv2.imread(image_path)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image_grey, (100, 100))
        images.append(resized_image)
        targets.append(0)
    except Exception as e:
        print("Exception occured as ",e)

Exception occured as  OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception occured as  OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception occured as  OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception occured as  OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception occured as  OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'

Exception occured as  OpenCV(4.3.0) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-21

In [ ]:
len(images), len(targets)

(3823, 3823)

In [ ]:
images

[array([[ 9,  9, 10, ..., 21, 27, 27],
        [ 8,  8,  7, ...,  8, 13, 15],
        [12, 10, 10, ...,  4,  4,  4],
        ...,
        [66, 67, 58, ...,  1,  0,  0],
        [66, 64, 53, ...,  2,  0,  0],
        [60, 56, 49, ...,  2,  0,  0]], dtype=uint8),
 array([[212, 189, 161, ...,  43,  57,  70],
        [196, 160, 123, ...,  32,  44,  54],
        [168, 123,  85, ...,  25,  33,  40],
        ...,
        [100, 102, 102, ..., 120, 125, 130],
        [101, 104, 105, ..., 118, 120, 121],
        [102, 107, 109, ..., 128, 127, 127]], dtype=uint8),
 array([[ 18,  23,  20, ...,  30,  30,  30],
        [ 21,  23,  19, ...,  31,  31,  30],
        [ 27,  29,  26, ...,  31,  30,  29],
        ...,
        [171, 168, 163, ..., 239, 239, 234],
        [175, 171, 165, ..., 232, 234, 235],
        [179, 174, 167, ..., 233, 233, 235]], dtype=uint8),
 array([[217, 218, 212, ..., 126, 141, 157],
        [216, 212, 198, ..., 103, 122, 148],
        [207, 199, 178, ...,  71,  96, 129],
       

In [ ]:
import numpy as np
images = np.array(images)/255.0

In [ ]:
images

array([[[0.03529412, 0.03529412, 0.03921569, ..., 0.08235294,
         0.10588235, 0.10588235],
        [0.03137255, 0.03137255, 0.02745098, ..., 0.03137255,
         0.05098039, 0.05882353],
        [0.04705882, 0.03921569, 0.03921569, ..., 0.01568627,
         0.01568627, 0.01568627],
        ...,
        [0.25882353, 0.2627451 , 0.22745098, ..., 0.00392157,
         0.        , 0.        ],
        [0.25882353, 0.25098039, 0.20784314, ..., 0.00784314,
         0.        , 0.        ],
        [0.23529412, 0.21960784, 0.19215686, ..., 0.00784314,
         0.        , 0.        ]],

       [[0.83137255, 0.74117647, 0.63137255, ..., 0.16862745,
         0.22352941, 0.2745098 ],
        [0.76862745, 0.62745098, 0.48235294, ..., 0.1254902 ,
         0.17254902, 0.21176471],
        [0.65882353, 0.48235294, 0.33333333, ..., 0.09803922,
         0.12941176, 0.15686275],
        ...,
        [0.39215686, 0.4       , 0.4       , ..., 0.47058824,
         0.49019608, 0.50980392],
        [0.3

In [ ]:
images.shape[0]

3823

In [ ]:
images = np.resize(images, (3823, 100, 100, 1))

In [ ]:
images

array([[[[0.03529412],
         [0.03529412],
         [0.03921569],
         ...,
         [0.08235294],
         [0.10588235],
         [0.10588235]],

        [[0.03137255],
         [0.03137255],
         [0.02745098],
         ...,
         [0.03137255],
         [0.05098039],
         [0.05882353]],

        [[0.04705882],
         [0.03921569],
         [0.03921569],
         ...,
         [0.01568627],
         [0.01568627],
         [0.01568627]],

        ...,

        [[0.25882353],
         [0.2627451 ],
         [0.22745098],
         ...,
         [0.00392157],
         [0.        ],
         [0.        ]],

        [[0.25882353],
         [0.25098039],
         [0.20784314],
         ...,
         [0.00784314],
         [0.        ],
         [0.        ]],

        [[0.23529412],
         [0.21960784],
         [0.19215686],
         ...,
         [0.00784314],
         [0.        ],
         [0.        ]]],


       [[[0.83137255],
         [0.74117647],
         [0.63

In [ ]:
from keras.utils import np_utils

In [ ]:
targets = np_utils.to_categorical(targets)

In [ ]:
targets

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(images, targets, test_size=0.25)

In [ ]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((2867, 100, 100, 1), (956, 100, 100, 1), (2867, 2), (956, 2))

In [ ]:
X_train[0]

array([[[0.50196078],
        [0.50196078],
        [0.50196078],
        ...,
        [0.62745098],
        [0.63137255],
        [0.63529412]],

       [[0.50980392],
        [0.50588235],
        [0.50588235],
        ...,
        [0.65882353],
        [0.65882353],
        [0.65882353]],

       [[0.50980392],
        [0.50980392],
        [0.50588235],
        ...,
        [0.67058824],
        [0.67058824],
        [0.66666667]],

       ...,

       [[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]],

       [[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]],

       [[1.        ],
        [1.        ],
        [1.        ],
        ...,
        [1.        ],
        [1.        ],
        [1.        ]]])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

In [ ]:
model = Sequential()

model.add(Conv2D(200, (3,3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(100, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
cp = ModelCheckpoint('model-best', verbose=0, save_best_only=True)
model.fit(X_train, Y_train, epochs = 15,  callbacks=[cp], validation_split=0.2)

Epoch 1/15
72/72 [==============================] - ETA: 0s - loss: 0.6592 - accuracy: 0.6005WARNING:tensorflow:From E:\Applications\Anaconda\lib\site-packages\tensorflow\python\training\tracking\tracking.py:111: Model.state_updates (from tensorflow.python.keras.engine.training) is deprecated and will be removed in a future version.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: model-best\assets
72/72 [==============================] - 126s 2s/step - loss: 0.6592 - accuracy: 0.6005 - val_loss: 0.5379 - val_accuracy: 0.7387
Epoch 2/15
72/72 [==============================] - 116s 2s/step - loss: 0.4941 - accuracy: 0.7658 - val_loss: 0.4158 - val_accuracy: 0.7962
Epoch 3/15
72/72 [==============================] - 102s 1s/step - loss: 0.3634 - accuracy: 0.8430 - val_lo

In [ ]:
import cv2
import numpy as np
import os

In [ ]:
from keras.models import load_model

In [ ]:
model = load_model('model-011.model')

In [ ]:
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
source = cv2.VideoCapture(0)

while 1:
    not_to_use, image = source.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detect.detectMultiScale(gray, 1.5,5)
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+w, x:x+w]
        resized_face = cv2.resize(face_roi, (100, 100))
        normalized_Face = resized_face/255
        reshaped_face = np.reshape(normalized_Face, (1, 100, 100, 1))
        result = model.predict(reshaped_face)[0]
        print("Probabilities : ",result)
        if result[0] > result[1]:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) 
            cv2.putText(image, "Safe", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

        if result[1] > result[0]:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "Not Safe", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
    cv2.imshow('Verzeo Project', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cv2.destroyAllWindows()
source.release()
    

Probabilities :  [0.03582711 0.9641729 ]
Probabilities :  [3.4439093e-10 1.0000000e+00]
Probabilities :  [1.6675661e-09 1.0000000e+00]
Probabilities :  [4.5958505e-08 1.0000000e+00]
Probabilities :  [2.3463494e-06 9.9999762e-01]
Probabilities :  [1.293629e-06 9.999987e-01]
Probabilities :  [0.00336625 0.9966337 ]
Probabilities :  [6.1608694e-07 9.9999940e-01]
Probabilities :  [2.7256638e-08 1.0000000e+00]
Probabilities :  [3.00283e-09 1.00000e+00]
Probabilities :  [1.2362906e-09 1.0000000e+00]
Probabilities :  [2.7405312e-08 1.0000000e+00]
Probabilities :  [7.941226e-06 9.999920e-01]
Probabilities :  [8.978950e-08 9.999999e-01]
Probabilities :  [4.0905519e-07 9.9999964e-01]
Probabilities :  [4.3013168e-07 9.9999952e-01]
Probabilities :  [3.6052885e-08 1.0000000e+00]
Probabilities :  [8.253059e-08 9.999999e-01]
Probabilities :  [6.984096e-08 9.999999e-01]
Probabilities :  [3.5514636e-06 9.9999642e-01]
Probabilities :  [1.5584647e-04 9.9984407e-01]
Probabilities :  [6.823547e-05 9.999318

Probabilities :  [4.3433452e-08 1.0000000e+00]
Probabilities :  [2.9137768e-08 1.0000000e+00]
Probabilities :  [1.4191896e-08 1.0000000e+00]
Probabilities :  [3.601948e-08 1.000000e+00]
Probabilities :  [3.0316582e-07 9.9999964e-01]
Probabilities :  [4.3203016e-08 1.0000000e+00]
Probabilities :  [5.4447526e-08 1.0000000e+00]
Probabilities :  [0.01514585 0.9848541 ]
Probabilities :  [1.821789e-08 1.000000e+00]
Probabilities :  [5.071797e-07 9.999995e-01]
Probabilities :  [2.936994e-08 1.000000e+00]
Probabilities :  [4.1789704e-08 1.0000000e+00]
Probabilities :  [2.3280823e-08 1.0000000e+00]
Probabilities :  [6.721294e-08 9.999999e-01]
Probabilities :  [3.263649e-08 1.000000e+00]
Probabilities :  [9.0323184e-08 9.9999988e-01]
Probabilities :  [4.842774e-08 1.000000e+00]
Probabilities :  [5.5720214e-08 1.0000000e+00]
Probabilities :  [3.3301774e-08 1.0000000e+00]
Probabilities :  [2.5665187e-08 1.0000000e+00]
Probabilities :  [2.6113746e-08 1.0000000e+00]
Probabilities :  [1.790168e-08 1.